In [ ]:
from __future__ import print_function
import numpy as np

from bqplot import (
    Axis, ColorAxis, LinearScale, DateScale, DateColorScale, OrdinalScale,
    OrdinalColorScale, ColorScale, Scatter, Lines, Figure, Tooltip
)

import ipywidgets as widgets

# Using one plot as a control for another

Although [bqplot](https://github.com/bloomberg/bqplot) is a versatile plotting package, one of its unique strengths is that the plot is a widget. Plot elements, like the position of points on the graph, can be monitored for changes like any other widget trait. 

The example in this notebook graphs a Fourier sine series using [bqplot](https://github.com/bloomberg/bqplot) to graph the series and a separate [bqplot](https://github.com/bloomberg/bqplot) to allow users to sest the amplitude of the terms in the series.

## Final widget will look like this:


<img src="images/plot-as-control.gif" width="50%">

## Define the function that calculates the Fourier series

Though in a real application the plot range and number of points should perhaps be configurable via a widget, for this example they are hard coded.

In [ ]:
def fourier_series(amplitudes):
    """
    Compute the fourier sine series given a set of amplitudes. The 
    period of the fundamental of the series is 1 and the series is 
    generated for two periods.
    """
    period = 1.0
    x = np.linspace(0, 2 * period, num=1000)
    y = np.sum(a * np.sin(2 * np.pi * (n + 1) * x / period) 
               for n, a in enumerate(amplitudes))

    return x, y

The number of Fourier components in the series should probably also be user-configurable; for the sake of simplicity it is hard coded here. We also define some test data so we can look at plot for setting the amplitudes before we connect it up to the plot of the Fourier series sum.

In [ ]:
N_fourier_components = 10
x_data = np.arange(N_fourier_components) + 1
y_data = np.random.uniform(low=-1, high=1, size=N_fourier_components)

## Create the amplitude control

We will create both this and the plot of the series using the "Grammar of Graphics" interface to bqplot. That makes it easier to treat the plot as a widget.

Please read through and execute the example below.

In [ ]:
# Start by defining a scale for each axis
sc_x = LinearScale()

# The amplitudes are limited to ±1 for this example...
sc_y = LinearScale(min=-1.0, max=1.0)

# You can create a Scatter object without supplying the data at this
# point. It is here so we can see how the control looks.
scat = Scatter(x=x_data, y=y_data, 
               scales={'x': sc_x, 'y': sc_y}, 
               colors=['orange'],
               # This is what makes this plot interactive
               enable_move=True)

# Only allow points to be moved vertically...
scat.restrict_y = True

# Define the axes themselves
ax_x = Axis(scale=sc_x)
ax_y = Axis(scale=sc_y, tick_format='0.2f', orientation='vertical')

# The graph itself...
amplitude_control = Figure(marks=[scat], axes=[ax_x, ax_y], 
                           title='Fourier amplitudes')

# This width is chosen just to make the plot fit nicely with 
# another. Change it if you want.
amplitude_control.layout.width = '400px'

# Let's see what this looks like...
amplitude_control

Try dragging the points on the graph around. Print the $y$-data from the plot (`scat.y`) and the original data below you they should be different.

## Set up some initial conditions

To test our sine series plot it is helpful to start with a simple-to-understand series: just the fundmental, with amplitude 1. 

In [ ]:
# Add some test data to make view the result
initial_amplitudes = np.zeros(10)
initial_amplitudes[0] = 1.0

## Create the plot of the Fourier series

As above, we create the plot using the Grammar of Graphics interface.

In [ ]:
lin_x = LinearScale()
lin_y = LinearScale()

# Note that here, unlike above, we do not set the initial data.
line = Lines(scales={'x': lin_x, 'y': lin_y}, colors=['orange'],
               enable_move=False)

ax_x = Axis(scale=lin_x)
ax_y = Axis(scale=lin_y, tick_format='0.2f', orientation='vertical')

result = Figure(marks=[line], axes=[ax_x, ax_y], 
                title='Fourier sine series',
                # Honestly, I just like the way the animation looks.
                # Value is in milliseconds.
                animation_duration=500)

# Size as you wish...
result.layout.width = '400px'

# Calculate the fourier series....
line.x, line.y = fourier_series(initial_amplitudes)

# Let's take a look!
result

### Set the amplitude control to match this initial case

Note that you can access the `scat` object from the `Figure` widget. Each line, scatter or other mark on the plot is in the list at `.marks`.

In [ ]:
amplitude_control.marks[0].y = initial_amplitudes

## You make the widget out of `amplitude_control` and `result`

See the animation above for a reminder of the target. Dragging the amplitudes will *not* change the sine series yet.

In [ ]:
# %load solutions/bqplot-as-control/box-widget.py

## Looks good, connect things up

Fill in the body of the function below, which will be observed by `amplitude_control`. You should call `fourier_series` and set the appropriate line properties.

In [ ]:
# %load solutions/bqplot-as-control/update_line.py
def update_line(change):
    pass


In [ ]:
# React to changes in the y value....
amplitude_control.marks[0].observe(update_line, names=['y'])